# Analisis de sentimiento con modelo transformer

# 0. Import

In [49]:
#Transformers
model_name = "finiteautomata/beto-sentiment-analysis"
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

import pandas as pd
import spacy
nlp = spacy.load("es_core_news_md")
from tqdm import tqdm # Barra de progreso

# 1. Leer dataset

In [50]:
FILE = "./noticias_losrios_2022_porcomuna.csv"

dataset = pd.read_csv(FILE)
dataset.drop('Unnamed: 0', axis=1, inplace=True)
dataset.head()

,id_news,country,media_outlet,url,title,text,date,search,Lago_Ranco,Los_Lagos,Futrono,Río_Bueno,La_Unión,Panguipulli,Paillaco,Corral,Máfil,Lanco,Mariquina,Valdivia
0,47314428.0,chile,suractual,https://www.suractual.cl/2022/01/24/ante-proye...,Ante proyecto Tierras Raras municipio y Unión ...,“Considerando la evolución que ha tenido el Pr...,2022-01-24,NaN,0,0,0,0,0,0,0,0,0,0,0,0
1,47719086.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/25/trac...,Tractocamión volcó esta mañana en puente Lloll...,El accidente se produjo pasadas las 06:00 AM y...,2022-01-25,NaN,0,0,0,0,1,0,0,0,0,0,0,0
2,47719110.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/27/gobe...,Gobernadores Regionales de Los Ríos y Los Lago...,La instancia buscó generar un trabajo colabora...,2022-01-27,NaN,0,1,0,1,0,0,0,0,0,0,0,0
3,47700339.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Alegan inocencia de acusadas por cruel asesina...,La defensa de las dos mujeres acusadas de enca...,2022-01-05,NaN,0,0,0,0,0,0,0,0,0,0,0,1
4,47700425.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Destacan ejecución del Programa Vínculos en Lo...,Para dar protección y acompañamiento a adultos...,2022-01-03,NaN,0,0,0,0,0,0,0,0,0,0,0,1


# 3. Clasificar sentimientos - ejemplos básicos

In [51]:
text1="Me gustan los gatos"
text2="No me gustan los perros"

In [52]:
sentiment_value = sentiment_pipeline(text1)
print(sentiment_value)

[{'label': 'POS', 'score': 0.9910821914672852}]


In [53]:
sentiment_value = sentiment_pipeline(text2)
print(sentiment_value)

[{'label': 'NEG', 'score': 0.9990025162696838}]


Cómo funciona el modelo Beto Sentiment Analysis: 
- https://huggingface.co/finiteautomata/beto-sentiment-analysis
- https://github.com/pysentimiento/pysentimiento
- https://arxiv.org/abs/2106.09462

Modelo BETO basado en BERT (2019): https://arxiv.org/abs/1810.04805

# 4. Clasificar sentimientos de noticias

In [54]:
text1=dataset["title"][7]+". "+dataset["text"][7]
text1

'CORE Los Ríos aprobó aumento de recursos para restauración de MAC Valdivia - Diario Laguino. El Consejo Regional de Los Ríos aprobó un aumento de recursos de $371 millones para la obra en construcción “Restauración y ampliación del Museo de Arte Contemporáneo” de Valdivia, tras hallazgos arqueológicos durante el avance de la obra. La solicitud de aumento de recursos fue realizada por la Dirección de Arquitectura del MOP, ejecutores del proyecto, debido a que este hito de protección patrimonial, implicaría recursos adicionales para los ítems de obras, consultorías y gastos administrativos. Al respecto, Consejero Regional y presidente de la Comisión de Infraestructura, Juan Taladriz manifestó que, “estamos hablando de una obra muy importante y emblemática para la ciudad de Valdivia, que permitirá potenciar y rescatar el patrimonio local. Creo que hoy en día la puesta en valor de la cultura y el patrimonio es igual de importante que cualquier otra materia, ya que nos entrega un sentido d

In [55]:
text2=dataset["title"][64]+". "+dataset["text"][64]
text2

'Vecinos denuncian estar aislados en Pilolcura tras crecida de estero  - Diario Laguino. La Junta de Vecinos de Pilolcura denuncia que hay más de 20 familias aisladas en el sector, debido a la crecida de un estero. Un hecho similar ocurrió el año pasado tras temporales registrados en la región. Así lo dio a conocer a Diario de Valdivia el presidente de dicha organización vecinal, Guillermo Anwandter, relatando que el único acceso que tienen al menos más de 20 familias del sector, se vio afectado por una crecida del estero Pilolcura tras las últimas lluvias que se han presentado en al región. Agregó que la situación también afecta a las personas que por leña festividad de año nuevo se trasladaron a Valdivia, sin tener ahora la posibilidad de regresar a sus hogares. El dirigente recordó que en junio del año pasado las lluvias provocadas por otro sistema frontal que afectó a la zona, se llevaron el puente que conectaba dicho acceso, por lo cual se procedió a una solución provisoria por pa

In [56]:
doc1 = nlp(text1)
doc2 = nlp(text2)

In [57]:
list_sentences1=[]
for sentence in doc1.sents:
    list_sentences1.append(sentence)
    
list_sentences2=[]
for sentence in doc2.sents:
    list_sentences2.append(sentence)

In [58]:
str(list_sentences1[0])

'CORE Los Ríos aprobó aumento de recursos para restauración de MAC Valdivia - Diario Laguino.'

In [59]:
sentiment_value = sentiment_pipeline(str(list_sentences1[0]))
print(sentiment_value)

[{'label': 'NEU', 'score': 0.9919497966766357}]


In [60]:
sentiment_value = sentiment_pipeline(str(list_sentences2[0]))
print(sentiment_value)

[{'label': 'NEG', 'score': 0.6180356740951538}]


# 5. Clasificar todas las noticias

In [62]:
for index, row in tqdm(dataset.iterrows(), desc='Análisis de sentimiento', total=dataset.shape[0]):
    
    sentiment_value = sentiment_pipeline(row['title'])
    
    # Insertamos en dataframe
    dataset.at[index, "prediction"] = sentiment_value[0].get('label')
    dataset.at[index, "score"] = sentiment_value[0].get('score')

Análisis de sentimiento: 100%|████████████| 15147/15147 [12:30<00:00, 20.20it/s]


In [63]:
dataset.to_csv("noticias_losrios_2022_porcomuna_sentimientos.csv")
dataset

,id_news,country,media_outlet,url,title,text,date,search,Lago_Ranco,Los_Lagos,...,La_Unión,Panguipulli,Paillaco,Corral,Máfil,Lanco,Mariquina,Valdivia,prediction,score
0,47314428.0,chile,suractual,https://www.suractual.cl/2022/01/24/ante-proye...,Ante proyecto Tierras Raras municipio y Unión ...,“Considerando la evolución que ha tenido el Pr...,2022-01-24,NaN,0,0,...,0,0,0,0,0,0,0,0,NEU,0.988633
1,47719086.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/25/trac...,Tractocamión volcó esta mañana en puente Lloll...,El accidente se produjo pasadas las 06:00 AM y...,2022-01-25,NaN,0,0,...,1,0,0,0,0,0,0,0,NEU,0.998064
2,47719110.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/27/gobe...,Gobernadores Regionales de Los Ríos y Los Lago...,La instancia buscó generar un trabajo colabora...,2022-01-27,NaN,0,1,...,0,0,0,0,0,0,0,0,NEU,0.931461
3,47700339.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Alegan inocencia de acusadas por cruel asesina...,La defensa de las dos mujeres acusadas de enca...,2022-01-05,NaN,0,0,...,0,0,0,0,0,0,0,1,NEU,0.973683
4,47700425.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Destacan ejecución del Programa Vínculos en Lo...,Para dar protección y acompañamiento a adultos...,2022-01-03,NaN,0,0,...,0,0,0,0,0,0,0,1,POS,0.536478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15142,50428917.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/educacion...,Admisión 2023: Caravana Técnico Profesional de...,Desde este lunes 12 y hasta el miércoles 14 de...,2022-12-13,NaN,0,1,...,0,0,0,0,0,0,0,0,NEU,0.979722
15143,50428943.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/emergenci...,Triple colisión en Los Lagos deja al menos un ...,Tres vehículos se vieron involucrados en un ac...,2022-12-07,NaN,0,1,...,0,0,0,0,0,0,0,0,NEG,0.716544
15144,50428975.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/opinion/2...,Columna de opinión: No quedó ni el sombrero - ...,“Somos en realidad sacos llenos de cosas inédi...,2022-12-09,NaN,0,0,...,0,0,0,0,0,0,0,0,NEG,0.977032
15145,50429002.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/emergenci...,Triste despertar en Los Lagos: comunidad lamen...,Un triste despertar ha tenido este domingo la ...,2022-12-04,NaN,0,1,...,0,0,0,0,0,0,0,0,NEG,0.997505
